In [51]:
import pandas as pd
import numpy as np
from datasets import Dataset

In [52]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Load Excel files

In [53]:
df_single = pd.read_excel("/content/drive/MyDrive/Emotion/emotaion_sentence.xlsx")
df_dial_train = pd.read_excel("/content/drive/MyDrive/Emotion/emotion_dialog_train.xlsx")
df_dial_val = pd.read_excel("/content/drive/MyDrive/Emotion/emotion_dialog_val.xlsx")

# 2. Sentence Preprocess

  - 놀람, 공포 제거

In [54]:
df_single = df_single[~df_single['Emotion'].isin(['공포', '놀람'])]
df_single['text'] = df_single['Sentence'].apply(
    lambda x: f"다음 문장을 읽고 감정을 추론하세요. \n문장 : {x}\n감정 : "
)
df_single['label'] = df_single['Emotion']

/tmp/ipython-input-54-4271316249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single['text'] = df_single['Sentence'].apply(
/tmp/ipython-input-54-4271316249.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single['label'] = df_single['Emotion']


In [55]:
df_single.head()

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468,text,label
11366,서해 수자원도 한류라서 그동안 불법조업 방치했나?,분노,NaN,NaN,NaN,NaN,NaN,다음 문장을 읽고 감정을 추론하세요. \n문장 : 서해 수자원도 한류라서 그동안 불...,분노
11367,니가 인간이가.,분노,NaN,NaN,NaN,NaN,NaN,다음 문장을 읽고 감정을 추론하세요. \n문장 : 니가 인간이가.\n감정 :,분노
11368,강아지를 때리고 유기하는것만 학대는 아닙니다!,분노,NaN,NaN,NaN,NaN,NaN,다음 문장을 읽고 감정을 추론하세요. \n문장 : 강아지를 때리고 유기하는것만 학대...,분노
11369,하여간 우리나라 교육엔 미래가 없어!!,분노,NaN,NaN,NaN,NaN,NaN,다음 문장을 읽고 감정을 추론하세요. \n문장 : 하여간 우리나라 교육엔 미래가 없...,분노
11370,채널에이 군부 개인거 인증하냐 동아일보야!!,분노,NaN,NaN,NaN,NaN,NaN,다음 문장을 읽고 감정을 추론하세요. \n문장 : 채널에이 군부 개인거 인증하냐 동...,분노


# 3. Dialog Preprocess(2개니까 function)
  - 당황 제거, 행복 -> 기쁨으로 변경

In [56]:
def preprocess_dialog(df):
  df = df[df['감정_대분류'] != '당황'].copy()
  df.loc[df['감정_대분류'] == '행복', '감정_대분류'] = '기쁨'

  def build(row):
    convo = []
    if pd.notnull(row['사람문장1']): convo.append(f"사람: {row['사람문장1']}")
    if pd.notnull(row['시스템문장1']): convo.append(f"시스템: {row['시스템문장1']}")
    if pd.notnull(row['사람문장2']): convo.append(f"사람: {row['사람문장2']}")
    if pd.notnull(row['시스템문장2']): convo.append(f"시스템: {row['시스템문장2']}")
    if pd.notnull(row['사람문장3']): convo.append(f"사람: {row['사람문장3']}")
    if pd.notnull(row['시스템문장3']): convo.append(f"시스템: {row['시스템문장3']}")
    prompt = "다음 대화를 읽고 화자의 감정을 추론하세요.\n" + "\n".join(convo) + "\n감정: "
    return pd.Series({'text': prompt, 'label': row['감정_대분류']})

  return df.apply(build, axis=1)

In [57]:
df_dial_train_proc = preprocess_dialog(df_dial_train)
df_dial_val_proc = preprocess_dialog(df_dial_val)

In [58]:
df_dial_train_proc['label'].unique()

array(['분노', '기쁨', '불안', '슬픔', '상처'], dtype=object)

In [59]:
df_dial_train_proc.head()

,text,label
0,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 일은 왜 해도 해도 끝이 없을...,분노
1,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 이번 달에 또 급여가 깎였어!...,분노
2,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 회사에 신입이 들어왔는데 말투...,분노
3,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 직장에서 막내라는 이유로 나에...,분노
4,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 얼마 전 입사한 신입사원이 나...,분노


In [60]:
df_dial_val_proc.head()

,text,label
0,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 이번 프로젝트에서 발표를 하는...,불안
1,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 회사에서 중요한 프로젝트를 혼...,불안
2,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 상사가 너무 무섭게 생겨서 친...,불안
3,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 이번에 힘들게 들어간 첫 직장...,불안
4,다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 직장에서 동료들이랑 관계가 안...,불안


# 4. Combine Train DF

In [61]:
df_train_combined = pd.concat([df_single[['text', 'label']], df_dial_train_proc], ignore_index=True)

# 5. Filter Labels

In [62]:
df_train_combined['label'].unique()

array(['분노', '슬픔', '중립', '행복', '혐오', '기쁨', '불안', '상처'], dtype=object)

In [63]:
df_train_combined['label'] = df_train_combined['label'].replace({'기쁨': '행복'})

In [64]:
df_train_combined['label'].unique()

array(['분노', '슬픔', '중립', '행복', '혐오', '불안', '상처'], dtype=object)

In [65]:
df_wounds = df_train_combined[df_train_combined['label'] == '상처']

print(f"wounds datas : {len(df_wounds)}")
print(df_wounds[['text', 'label']].head())

wounds datas : 9143
                                                    text label
27249  다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 면접관에게 완전히 속았어. 면...    상처
27250  다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 지인이 취업시켜준다면서 사기를...    상처
27251  다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 우리 부모님은 나를 버린 거나...    상처
27252  다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 계속 취업이 안 되니까 버려진...    상처
27264  다음 대화를 읽고 화자의 감정을 추론하세요.\n사람: 나 너무 우울해. 회사에서 큰...    상처


In [66]:
df_train_combined = df_train_combined[df_train_combined['label'] != '상처'].reset_index(drop=True)

In [67]:
df_train_combined['label'].unique()

array(['분노', '슬픔', '중립', '행복', '혐오', '불안'], dtype=object)

In [68]:
labels = ['기쁨', '슬픔', '중립', '혐오', '분노', '불안']
df_train_combined = df_train_combined[df_train_combined['label'].isin(labels)].reset_index(drop=True)
df_dial_val_proc = df_dial_val_proc[df_dial_val_proc['label'].isin(labels)].reset_index(drop=True)

# 6. labl2id

In [69]:
label2id = {label: idx for idx, label in enumerate(sorted(labels))}

df_train_combined['label_id'] = df_train_combined['label'].map(label2id)
df_dial_val_proc['label_id'] = df_dial_val_proc['label'].map(label2id)

# 7. 파일들 내보내기

In [70]:
train_dataset = Dataset.from_pandas(df_train_combined[['text','label_id']].reset_index(drop=True))
train_dataset

Dataset({
    features: ['text', 'label_id'],
    num_rows: 48796
})

In [71]:
df_train_combined['label_id'] = df_train_combined['label_id'].astype(int)

df_train_combined[['text','label_id']].to_json(
    "/content/drive/MyDrive/Emotion/emotion_train.jsonl",
    orient='records',
    lines=True,
    force_ascii=False
)
print("train_dataset saved~!")

train_dataset saved~!


In [72]:
df_dial_val_proc[['text','label_id']].to_json(
    "/content/drive/MyDrive/Emotion/emotion_val.jsonl",
    orient='records',
    lines=True,
    force_ascii=False
)
print("val_dataset saved~!")

val_dataset saved~!


In [73]:
import json

with open("/content/drive/MyDrive/Emotion/label2id.json", "w", encoding="utf-8") as f:
  json.dump(label2id, f, ensure_ascii=False, indent=2)

print("label2id saved~!")

label2id saved~!


In [74]:
label2id

{'기쁨': 0, '분노': 1, '불안': 2, '슬픔': 3, '중립': 4, '혐오': 5}